### Imports

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

import config
from src import data_ingestion
from src import data_providers
from src import benchmark
from src import portfolio

### Load Data

In [2]:
# --- Load Data ---
master_log = data_ingestion.create_master_log([
    config.INPUT_DATA_DIR / "us_mkt_transactions.csv",
    config.INPUT_DATA_DIR / "exus_mkt_transactions.csv"
]).set_index('Date')

master_log

Successfully loaded log: us_mkt_transactions.csv
Successfully loaded log: exus_mkt_transactions.csv


,Type,Symbol,Quantity,Price,Amount,Commission,Currency,Description,Market,Source
Date,,,,,,,,,,
2023-02-16,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,ID: 37fdafdc-d707-42fa-ba74-98b30cf9ab2a - DT2...,US Market,Sarwa Trade
2023-02-17,buy,PERI,0.407270,33.340000,-13.58,NaN,USD,Trade Entry,US Market,Sarwa Trade
2023-02-17,buy,TSLA,0.202666,201.020000,-40.74,NaN,USD,Trade Entry,US Market,Sarwa Trade
2023-02-17,buy,VOO,0.218967,372.070000,-81.47,NaN,USD,Trade Entry,US Market,Sarwa Trade
2023-03-29,Net Dividend,VOO,NaN,NaN,0.24,NaN,USD,"Cash DIV @ 1.4874, Pos QTY: 0.218966527, Rec D...",US Market,Sarwa Trade
...,...,...,...,...,...,...,...,...,...,...
2025-07-24,buy,BRK-B,0.124000,481.935484,-59.76,1.0,USD,Purchase of BRK-B,US Market,Sarwa Trade - Interim
2025-07-24,buy,BTC-USD,0.000162,120615.384615,-19.60,NaN,USD,NaN,US Market,Sarwa Crypto
2025-07-24,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,Bank Deposit,US Market,Sarwa Trade - Interim


### `data_providers.py` Test

In [3]:
# --- 1. Initial Symbol Assessment ---
unique_symbols = master_log['Symbol'].dropna().unique()
found_df, missing_list = data_providers.assess_symbols_with_cache(unique_symbols)

Checking new symbol 'PERI' with yfinance...
Checking new symbol 'TSLA' with yfinance...
Checking new symbol 'VOO' with yfinance...
Checking new symbol 'UAE' with yfinance...
Checking new symbol 'XLK' with yfinance...
Checking new symbol 'BTC-USD' with yfinance...
Checking new symbol 'VOOG' with yfinance...
Checking new symbol 'NOBL' with yfinance...
Checking new symbol 'SPY' with yfinance...
Checking new symbol 'VHT' with yfinance...
Checking new symbol 'NVDA' with yfinance...
Checking new symbol 'SHOP' with yfinance...
Checking new symbol 'LUX' with yfinance...
Checking new symbol 'META' with yfinance...
Checking new symbol 'MSFT' with yfinance...
Checking new symbol 'IBB' with yfinance...
Checking new symbol 'SPHY' with yfinance...
Checking new symbol 'STIP' with yfinance...
Checking new symbol 'USRT' with yfinance...
Checking new symbol 'BA' with yfinance...
Checking new symbol 'ADBE' with yfinance...
Checking new symbol 'BKNG' with yfinance...
Checking new symbol 'CAT' with yfinanc

In [4]:
print("✅ The following symbols were automatically identified using yfinance:")
found_df

✅ The following symbols were automatically identified using yfinance:


,Name,Type,Exchange,Currency,Industry,Sector,DataProvider
PERI,Perion Network Ltd.,equity,NasdaqGS,USD,Internet Content & Information,Communication Services,yfinance
TSLA,"Tesla, Inc.",equity,NasdaqGS,USD,Auto Manufacturers,Consumer Cyclical,yfinance
VOO,Vanguard S&P 500 ETF,etf,NYSEArca,USD,None,None,yfinance
UAE,iShares MSCI UAE ETF,etf,NasdaqGM,USD,None,None,yfinance
XLK,The Technology Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,yfinance
...,...,...,...,...,...,...,...
JPMB,JPMorgan USD Emerging Markets Sovereign Bond ETF,etf,NYSEArca,USD,None,None,yfinance
SMOT,VanEck Morningstar SMID Moat ETF,etf,Cboe US,USD,None,None,yfinance
XLV,The Health Care Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,yfinance
BX,Blackstone Inc.,equity,NYSE,USD,Asset Management,Financial Services,yfinance


In [5]:
print("❌ The following symbols were not found on yfinance:")
missing_list

❌ The following symbols were not found on yfinance:


['ALDAR', 'BURJEEL', 'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY']

In [6]:
# --- 2. User Verification Step ---
# In this cell, the user reviews the table above and lists any symbols
# that yfinance identified incorrectly.
incorrectly_identified_symbols = [
    "AMR", "ASM"
]

# --- 3. Update Cache Based on User Input ---
data_providers.mark_symbols_as_user_defined(incorrectly_identified_symbols)

Updating cache for incorrectly identified symbols: ['AMR', 'ASM']
Cache updated successfully.


In [ ]:
# --- 4. Get Final Lists for Data Fetching ---
# Reload the cache to get the final, corrected state
final_cache = data_providers._load_json_cache(config.METADATA_CACHE)

symbols_from_yfinance = [s for s, d in final_cache.items() if d.get('DataProvider') == 'yfinance']
symbols_from_user = [s for s, d in final_cache.items() if d.get('DataProvider') in ['missing', 'user_defined']]

print(f"\nProceeding to fetch price data from yfinance for: {symbols_from_yfinance}")
print(f"Expecting user-provided price data & metadata for: {symbols_from_user}")


Proceeding to fetch price data from yfinance for: ['PERI', 'TSLA', 'VOO', 'UAE', 'XLK', 'BTC-USD', 'VOOG', 'NOBL', 'SPY', 'VHT', 'NVDA', 'SHOP', 'LUX', 'META', 'MSFT', 'IBB', 'SPHY', 'STIP', 'USRT', 'BA', 'ADBE', 'BKNG', 'CAT', 'EL', 'DE', 'COIN', 'OCSL', 'TPVG', 'GSBD', 'LLY', 'NVO', 'CLM', 'CEFS', 'GOF', 'SPXL', 'TQQQ', 'BRW', 'MOAT', 'AMZN', 'INDA', 'KSA', 'SPSM', 'VBK', 'IJR', 'PAK', 'MCHI', 'EWJ', 'MBB', 'ARKK', 'IPO', 'VXUS', 'XLF', 'ARM', 'ROM', 'RYAAY', 'APO', 'KKR', 'JPMB', 'SMOT', 'XLV', 'BX', 'BRK-B']
Expecting user-provided price data & metadata for: ['ALDAR', 'BURJEEL', 'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY', 'ASM', 'AMR']


In [8]:
# --- 5. Generate Metadata Template for User ---
# This new step creates the necessary file for the user to complete.
data_providers.create_user_metadata_template(master_log, symbols_from_user)


📝 Created/updated metadata template for: ['ALDAR', 'BURJEEL', 'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY', 'ASM', 'AMR']
Please fill in the details in the file: /Users/rehabnaeem/Developer/investment-portfolio-test/data/user-data/metadata.json


### Project Variables

In [ ]:
import pandas as pd
pd.read_csv(config.DATA_DIR / "market-data/")

In [ ]:
# --- Initialize Project Variables ---
start_date, end_date, date_range, last_market_day = config.project_dates(master_log.index)

# User-defined constants are accessed directly
benchmark_index = config.BENCHMARK_INDEX
tax_rate = config.TAX_RATE

In [ ]:
start_date
end_date
last_market_day

In [ ]:
data_providers.yf_hist("STIP", start_date, last_market_day)

### `portfolio.py` Test

In [ ]:
# In your main script or notebook (e.g., src/portfolio.py)

# Get a unique list of every stock and its designated market from the log
symbol_market_pairs = master_log[['Symbol', 'Market']].dropna().drop_duplicates()

# Initialize empty DataFrames for price and split data
raw_splits = pd.DataFrame(index=date_range, columns=symbols)
holdings['price'] = pd.DataFrame(index=date_range, columns=symbols)

# Loop through the unique pairs to fetch data using the correct provider
for _, row in symbol_market_pairs.iterrows():
    symbol = row['Symbol']
    market = row['Market']

    print(f"Fetching data for {symbol} on {market}...")
    hist = None

    if market == 'US Market':
        # Use yfinance for all US market transactions
        hist = data_providers.yfinance_hist(symbol, start_date, end_date, last_market_day)
    else:
        # Use Twelve Data for all other markets
        # The 'Market' value (e.g., "ADX", "DFM") is used as the exchange code
        hist = data_providers.get_twelve_data_history(config.API_KEY_TWELVE_DATA, symbol, exchange=market)

    # Populate the main dataframes with the fetched data
    if hist is not None and not hist.empty:
        # Use .reindex() to align the data with your full date_range
        holdings['price'][symbol] = hist['Close'].reindex(date_range).ffill()
        raw_splits[symbol] = hist['Stock Splits'].reindex(date_range)

# Fill any remaining NaNs after fetching data for all symbols
raw_splits = raw_splits.fillna(0.0)

# Now, you can proceed with your existing holdings calculation loop
# ...